# LWE and RLWE encryption
## LWE encryption

In FHEW-like HE, we use both LWE and RLWE encryption.
LWE is for a ciphertext and RLWE is for the core part of bootstrapping so called *blind rotation*.

We define LWE encryption as follows
$$\texttt{LWE}_{\vec{s}, n, q}(m) = (\beta, \vec{\alpha}) \in \mathbb{Z}_q^{n+1},$$
where $\beta = m + e - \left< \vec{\alpha}, \vec{s} \right> \in \mathbb{Z}_q$, and $\vec{s} \leftarrow \chi_{key}$ is a secret key and $\vec{e} \leftarrow \chi_{err}$ is a added noise for security.
$\vec{\alpha}$ is unifromly sampled in $\mathbb{Z}_q^n$.

Here, we choose $\chi_{key}$ as binary distribution and $\chi_{err}$ as a Gaussian distribution with standard deviation $3.2$.

Let's make the encryption method. We use pytorch for easy and fast implementation.

In [1]:
import torch
import numpy as np


We use parameter sets $(n, q, \sigma) = (512, 2048, 3.2)$.

In [2]:
stddev = 3.2
n = 512
q = 2048

Following are generator of key and error.

NOTE: Those generators are not secure. You should **NOT** use them in practice.

In [3]:
def keygen(dim):
    return torch.randint(2, size = (dim,))
def errgen(stddev):
    e = torch.round(stddev*torch.randn(1))
    e = e.squeeze()
    return e.to(torch.int)
def uniform(dim, modulus):
    return torch.randint(modulus, size = (dim,))

We first generate the secret key $\vec{s}$.

In [4]:
s = keygen(n)
s

tensor([0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
        0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
        0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,

To encrypt, we need random part $\vec{\alpha}$.

In [5]:
alpha = uniform(n, q)
alpha

tensor([1273, 1743, 1642, 1249, 1307,  252,   52, 1288, 1809, 1828, 1185, 1945,
        1402, 1011,  925, 1596, 1372, 1363,  939,  900,  713,  240, 2023, 1145,
         823, 1947, 1770, 1123,  753,  147,  302, 1480, 1391, 1522,  549,  833,
        1066,  106,  603,  351,  805, 1325,  842, 1883, 1857, 1029, 1684, 1257,
        1772, 1578, 1345,  839,    9,  862, 1650, 1682,  301, 1450, 1203, 1791,
         284, 1263, 1711,  923,  534,   99, 1009, 1755, 2002, 1203,  854, 1365,
        2042, 1528, 1518, 2028, 1828,  102, 1920, 1607,  412, 1250,  739, 1608,
        1775,   13,  463,  448, 1517, 1708, 1892, 1859, 1216, 1912, 1876,  442,
         787, 1680, 1395, 1011,  901,  559, 1008,  152, 2005, 1458, 1173, 1883,
        1534,  613, 1623, 1209,  172,  443, 1421,   97,  527,  457,  748,   99,
          41,  145, 1653, 1315, 1423, 1054, 1767,  885,  948, 1786,  388, 1310,
         607,  960, 1606,  713,  956,  468,  628, 1778, 1475, 1359,  341, 1624,
        2037, 1221, 1203,  139,  769, 16

We calaulate $\beta = m + e - \left< \vec{\alpha}, \vec{s} \right>$ for encryption.

Let the message we are encrypting is a binary value e,g, $m = 1$ here.

In [6]:
m = 1

beta = m - torch.dot(alpha, s)
e = errgen(stddev)
beta += e

beta %= q

beta

tensor(7)

By *LWE assumption* $\beta$ should look like a random value.
Now the pair $(\beta, \vec{\alpha})$ is our ciphertext.

Let's decrypt the ciphertext above.

As $\beta = m + e - \left< \vec{\alpha}, \vec{s} \right>$, we can find $m + e = \beta + \left< \vec{\alpha}, \vec{s} \right>$.

In [7]:
m_decrypted = beta + torch.dot(alpha, s)
m_decrypted %= q
m_decrypted

tensor(0)

If you are very lucky, you might get the decrypted value.
```
>>> m_decrypted
tensor(1)
```
But, if you run the code once, you will get other value.
Note here that we get $m+e$ by decryption, *not the exact value* $m$.

To make our message safe from the error, we can multiply certain *scaling factor* to our message.

Here, let's multiply $q/4$, and encrypt/decrypt again.


In [8]:
m = 1
# multiply scaling factor q/4 
m *= q//4

beta = m - torch.dot(alpha, s)
e = errgen(stddev)
beta += e
beta %= q

m_decrypted = beta + torch.dot(alpha, s)
m_decrypted %= q

m_decrypted

tensor(512)

We got a value near $m \cdot q/4 = 512$.
Division by $q/4$ and rounding will give us original message.

In [9]:
# rescale the message
m_decrypted = m_decrypted.to(torch.float)
m_decrypted /= q/4.
m_decrypted = torch.round(m_decrypted)
m_decrypted.to(torch.int)


tensor(1, dtype=torch.int32)

Decryption is successful!


### LWE encryption function

The LWE ciphertext is a pair $(\beta, \vec{\alpha})$.

We define the encryptor as follows.

In [10]:
def encryptLWE(message, dim, modulus, key):
    alpha = uniform(dim, modulus)

    beta = message * modulus//4 - torch.dot(alpha, key)
    e = errgen(stddev)
    beta += e
    beta %= modulus

    return (beta, alpha)


ct = encryptLWE(1, n, q, s)
ct    

(tensor(1090),
 tensor([ 813,  427, 1442, 1080,  773,  305,  513, 1108,  860, 1593,  765,  967,
          453, 1686, 1271, 1699, 1192, 1567, 1339, 1099, 1867,  638, 2038, 1655,
         1752,   22, 1630,  517,  453, 1724,   27,  733,  681, 1184, 1270, 1813,
          561,  788, 1469, 1096, 1737, 1501,  104, 1299, 1763, 1847, 1767, 1450,
          694, 1200,  884, 1696, 2018, 1707,  429, 1499,  454, 1903, 1842, 1956,
         1335, 1261,  141, 1770, 1662,  734,  836, 1805,    8,  209, 1884,  505,
         1593, 1508, 1635, 1095, 1331, 1004, 1392,  350, 1072,  125, 1651,  638,
         2036, 1821, 2030,  943, 1309, 1453, 1887,  229, 1817,  833, 1164,  433,
         1632, 1459, 1786,  135,  922, 1519,   86,  646, 1487, 1698,  461, 1527,
         1354, 1421, 1902, 1626,  885, 1996,  733, 1744,    6, 1569, 1753,  485,
         1820,  658, 1129,  659, 1139, 1058, 1320, 1703, 1949,  345,  783,  406,
          415, 1356,  554, 1834, 1002, 1366, 1917, 1997, 1462,  713,  778, 2001,
         1418

We can also define decryption.

In [11]:
def decryptLWE(ct, key, modulus):
    beta, alpha = ct
    m_dec = beta + torch.dot(alpha, key)
    m_dec %= modulus

    m_dec = m_dec.to(torch.float)
    m_dec /= modulus/4.
    m_dec = torch.round(m_dec)
    return m_dec.to(torch.int)

decryptLWE(ct, s, q)   

tensor(1, dtype=torch.int32)